# CRIM Intervals:  Presentation Types

### Note:  Still Under Development!:

#### Import Music Files

* If you are exploring pieces from CRIM, importing simply involves providing the CRIM URL of the MEI file:  
    * **`piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')`**

* But you can also use the Notebook with any MEI, MusicXML, or MIDI file of your own. You can easily do this when you run the Notebooks on Jupyter Hub, you will also find a folder called **`Music_Files`**.  Upload the file here, then provide the path to that file: 
    * **`piece = importScore('Music_Files/My_File_Name.mei')`**.  

#### Save outputs as CSV or Excel

* The Jupyter Hub version of these Notebooks also provides a folder called **`saved_csv`**.  You can save **csv** files of any data frame there with this command: 
    * **`notebook_data_frame_name.to_csv('saved_csv/your_file_title.csv')`**.
* If you prefer **Excel** documents (which are better for anything with a complex set of columns or hierarhical index), use **ExcelWriter**.  In the following code, you will need to provide these commands:
    * **`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')`**
* Now convert your dataframe to Excel
    * **`frame_name.to_excel(writer, sheet_name='Sheet1')`**
* And finally save the new file to the folder here in the Notebook:
    * **`writer.save()`**

Put the following code to a new cell and update the frame_name and file_name:

`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')` <br>
`frame_name.to_excel(writer, sheet_name='Sheet1')` <br>
`writer.save()` <br>


## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [ ]:
import intervals
from intervals import * 
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy

# You should change 'test' to your preferred folder.
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    

        

In [ ]:
# New version to include in main_objs code
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    tone_list = []
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints
        
        


## B. Importing a Piece and Run Imitation Classifier

In [ ]:
piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')
points = piece.getPoints()


#### B.2 Finding Time and Entry Interval Details

*  This is temporary code, but please run it.

In [33]:

df3 = points

# make lists of voices and offsets and put them in dfs to be merged

voice_groups = df3.groupby('sub_group_id')['part'].apply(list)
voice_groups_df = voice_groups.reset_index(name = 'voice_list')

offset_groups = df3.groupby('sub_group_id')['start_offset'].apply(list)
offset_groups_df = offset_groups.reset_index(name = 'offset_list')

meas_beat_groups = df3.groupby('sub_group_id')['start'].apply(list)
meas_beat_groups_df = meas_beat_groups.reset_index(name = 'measure_beat_list')


# keep just the first entry in each point
df3 = df3[df3["entry_number"] == 1]

# merge the dfs of offsets and voices into the main list of points
# zip the lists of offsets and voices together as tuples
# find the offset = time intervals

df5 = pd.merge(df3, voice_groups_df,
how='inner', on='sub_group_id')

df6 = pd.merge(df5, offset_groups_df,
how='inner', on='sub_group_id')

df7 =  pd.merge(df6, meas_beat_groups_df,
how='inner', on='sub_group_id')

df7["time_intervals"] = df7['offset_list'].apply(lambda x: numpy.diff(x))

df8 = df7[["piece_title", "pattern_matched", "predicted_type", 'start_offset', "measure_beat_list", "offset_list", "time_intervals", "voice_list"]].copy()

df8['tone_coordinates'] = df8.apply(lambda row: list(zip(row.offset_list, row.voice_list)), axis='columns')

#  apply the function to find the entry intervals
# remove cols no long needed

df8['entry_intervals'] = df8['tone_coordinates'].apply(lambda x: find_entry_int_distance(x, piece))
df8.drop(columns=['offset_list', 'tone_coordinates'])

,piece_title,pattern_matched,predicted_type,start_offset,measure_beat_list,time_intervals,voice_list,entry_intervals
0,Je suis déshéritée,"(1, 1, 3, 1)",Fuga,4.0,"[1/3.0, 3/1.0]",[12.0],"[Tenor, Superius]",[P8]
1,Je suis déshéritée,"(1, 3, 1, 2)",Fuga,8.0,"[2/1.0, 3/3.0]",[12.0],"[Tenor, Superius]",[P8]
2,Je suis déshéritée,"(3, 1, 2, -4)",Fuga,10.0,"[2/2.0, 3/4.0]",[12.0],"[Tenor, Superius]",[P8]
3,Je suis déshéritée,"(2, -2, -2, 2)",Fuga,14.0,"[2/4.0, 6/2.5]",[29.0],"[Contratenor, Bassus]",[M-9]
4,Je suis déshéritée,"(-2, -2, 2, -2)",Fuga,16.0,"[3/1.0, 6/1.5]",[25.0],"[Contratenor, Bassus]",[P-8]
5,Je suis déshéritée,"(1, 1, -2, -2)",Fuga,34.0,"[5/2.0, 7/2.0]",[16.0],"[Tenor, Superius]",[P8]
6,Je suis déshéritée,"(1, -2, -2, -2)",Fuga,36.0,"[5/3.0, 7/3.0]",[16.0],"[Tenor, Superius]",[P8]
7,Je suis déshéritée,"(-2, -2, -2, 2)",Fuga,38.0,"[5/4.0, 7/4.0]",[16.0],"[Tenor, Superius]",[P8]
8,Je suis déshéritée,"(-2, -2, 2, -4)",Fuga,40.0,"[6/1.0, 8/1.0]",[16.0],"[Tenor, Superius]",[P8]
9,Je suis déshéritée,"(1, 1, 3, 1)",Fuga,64.0,"[9/1.0, 9/4.0, 11/1.0]","[6.0, 10.0]","[Tenor, Contratenor, Superius]","[P5, P4]"


In [34]:
classified_with_intervals = df8
classified_with_intervals.head()

,piece_title,pattern_matched,predicted_type,start_offset,measure_beat_list,offset_list,time_intervals,voice_list,tone_coordinates,entry_intervals
0,Je suis déshéritée,"(1, 1, 3, 1)",Fuga,4.0,"[1/3.0, 3/1.0]","[4.0, 16.0]",[12.0],"[Tenor, Superius]","[(4.0, Tenor), (16.0, Superius)]",[P8]
1,Je suis déshéritée,"(1, 3, 1, 2)",Fuga,8.0,"[2/1.0, 3/3.0]","[8.0, 20.0]",[12.0],"[Tenor, Superius]","[(8.0, Tenor), (20.0, Superius)]",[P8]
2,Je suis déshéritée,"(3, 1, 2, -4)",Fuga,10.0,"[2/2.0, 3/4.0]","[10.0, 22.0]",[12.0],"[Tenor, Superius]","[(10.0, Tenor), (22.0, Superius)]",[P8]
3,Je suis déshéritée,"(2, -2, -2, 2)",Fuga,14.0,"[2/4.0, 6/2.5]","[14.0, 43.0]",[29.0],"[Contratenor, Bassus]","[(14.0, Contratenor), (43.0, Bassus)]",[M-9]
4,Je suis déshéritée,"(-2, -2, 2, -2)",Fuga,16.0,"[3/1.0, 6/1.5]","[16.0, 41.0]",[25.0],"[Contratenor, Bassus]","[(16.0, Contratenor), (41.0, Bassus)]",[P-8]


### Get Presentation Types with Interact


In [ ]:
@interact
def get_points(duration_type=['real', 'incremental'], interval_type=["generic", "semitone"], match_type=["close", "exact"], min_exact_matches=[2, 3, 4, 5, 6], min_close_matches=[2, 3, 4, 5, 6], close_distance=[1, 2, 3, 4, 5, 6], vector_size=[3, 2, 4, 5, 6, 7], increment_size=[4, 2, 1], forward_gap_limit=[40, 20, 10], backward_gap_limit=[40, 20, 10], min_sum_durations=[10, 5, 20], max_sum_durations=[30, 50, 15], offset_difference_limit=[500, 100, 50]):
    points = piece.getPoints(duration_type=duration_type, interval_type=interval_type, match_type=match_type, min_exact_matches=min_exact_matches, min_close_matches=min_close_matches, close_distance=close_distance, vector_size=vector_size, increment_size=increment_size, forward_gap_limit=forward_gap_limit, backward_gap_limit=backward_gap_limit, min_sum_durations=min_sum_durations, max_sum_durations=max_sum_durations, offset_difference_limit=offset_difference_limit)
    return points
    points

### Batch Search for Presentation Types

List all pieces in quotation marks, and separated by commas

#### Now access the results:  

* First:  `list_of_dfs[0]`
* Second: `list_of_dfs[1]`

Or:

`for result in list_of_dfs:
    print(result)`

In [45]:
list_of_pieces = ['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei']
corpus = CorpusBase(list_of_pieces)
func = ImportedPiece.getPoints  # <- NB there are no parentheses here
list_of_dfs = corpus.batch(func)


Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Finding close matches...
223 melodic intervals had more than 3 exact or close matches.

Memoized piece detected.
Finding close matches...
60 melodic intervals had more than 3 exact or close matches.



### Filter the Points of Imitation According to the Nearby Cadences

* If the point begins within 8 beats of the end of any cadence, keep it.  Otherwise, omit it.
* The threshold around the cadence is adjustable:  `ln = item - 8
un = item + 8`

In [40]:
piece = importScore('https://crimproject.org/mei/CRIM_Model_0009.mei')
cads = piece.classifyCadences()
# points = piece.getPoints()
cads_index = cads.index.to_list()
cads_index
cads_index_rounded = [round(x) for x in cads_index]
cads_index_rounded

final_filter_list = []
for item in cads_index_rounded:
    ln = item - 8
    un = item + 8
    for x in range(ln, un):
        final_filter_list.append(x)
filtered_pts = classified_with_intervals.loc[classified_with_intervals[('start_offset')].isin(final_filter_list)]
first_event = classified_with_intervals.iloc[0]
filtered_pts = filtered_pts.append(first_event)
filtered_pts.sort_index(inplace = True)
filtered_pts



Memoized piece detected.


,piece_title,pattern_matched,predicted_type,start_offset,measure_beat_list,offset_list,time_intervals,voice_list,tone_coordinates,entry_intervals
0,Je suis déshéritée,"(1, 1, 3, 1)",Fuga,4.0,"[1/3.0, 3/1.0]","[4.0, 16.0]",[12.0],"[Tenor, Superius]","[(4.0, Tenor), (16.0, Superius)]",[P8]
8,Je suis déshéritée,"(-2, -2, 2, -4)",Fuga,40.0,"[6/1.0, 8/1.0]","[40.0, 56.0]",[16.0],"[Tenor, Superius]","[(40.0, Tenor), (56.0, Superius)]",[P8]
9,Je suis déshéritée,"(1, 1, 3, 1)",Fuga,64.0,"[9/1.0, 9/4.0, 11/1.0]","[64.0, 70.0, 80.0]","[6.0, 10.0]","[Tenor, Contratenor, Superius]","[(64.0, Tenor), (70.0, Contratenor), (80.0, Su...","[P5, P4]"
10,Je suis déshéritée,"(1, 3, 1, 2)",Fuga,68.0,"[9/3.0, 10/1.0, 11/3.0]","[68.0, 72.0, 84.0]","[4.0, 12.0]","[Tenor, Contratenor, Superius]","[(68.0, Tenor), (72.0, Contratenor), (84.0, Su...","[P5, P4]"
11,Je suis déshéritée,"(3, 1, 2, -4)",Fuga,70.0,"[9/4.0, 11/4.0]","[70.0, 86.0]",[16.0],"[Tenor, Superius]","[(70.0, Tenor), (86.0, Superius)]",[P8]
16,Je suis déshéritée,"(-2, 2, 2, 2)",Fuga,106.0,"[14/2.0, 14/4.0]","[106.0, 110.0]",[4.0],"[Tenor, Contratenor]","[(106.0, Tenor), (110.0, Contratenor)]",[P4]
17,Je suis déshéritée,"(1, 2, 3, 1)",Fuga,134.0,"[17/4.0, 19/2.0]","[134.0, 146.0]",[12.0],"[Tenor, Superius]","[(134.0, Tenor), (146.0, Superius)]",[P8]
20,Je suis déshéritée,"(4, -2, -2, -2)",Fuga,190.0,"[24/4.0, 25/3.0]","[190.0, 196.0]",[6.0],"[Tenor, Bassus]","[(190.0, Tenor), (196.0, Bassus)]",[P-5]


### Save the Filtered Points as CSV files

In [50]:


for result in list_of_dfs:
    title = result["piece_title"][0]
    # print(title)
    fn = 'saved_csv/' + title + '_Filtered_Points.csv'
    result.to_csv(fn)